## Setup

In [2]:
from langchain_ollama.chat_models import ChatOllama
llm = ChatOllama(model = "llama3.1:8b-instruct-fp16")

In [3]:
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model = "llama3.1:8b-instruct-fp16")

#### Milvus vs. In-memory Vector Store

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [4]:
from langchain_milvus import Milvus

vector_store = Milvus(embedding_function=embeddings, auto_id = True) # Server on port 19530

In [5]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
os.environ["LANGCHAIN_PROJECT"] = "rag"

 ········


#### Dependencies

In [9]:
%%capture --no-stderr
%pip install --upgrade --quiet langgraph langchain-community beautifulsoup4

&emsp;

ADD GITIGNORE & FIX FILTER - NOT CORRECT CODE FOR MILVUS!!!